# Variational Autoencoder for Latent Factor Discovery

This notebook demonstrates how to use Variational Autoencoders (VAE) to discover latent factors in stock returns, similar to how PCA finds principal components but with non-linear relationships.

## Key Concepts
- **Autoencoder**: Neural network that learns compressed representations
- **Variational Autoencoder (VAE)**: Probabilistic autoencoder with regularized latent space
- **Latent Factors**: Hidden variables that explain cross-sectional return variation

## Requirements
```bash
pip install torch numpy pandas matplotlib yfinance scikit-learn
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Generate Synthetic Stock Returns with Factor Structure

We generate synthetic returns that follow a factor model:
$$r_i = \beta_i^{\top} f + \epsilon_i$$

where $f$ are latent factors and $\beta_i$ are factor loadings.

In [ ]:
def generate_factor_returns(n_stocks=100, n_days=1000, n_factors=5):
    """Generate synthetic returns with factor structure"""
    # Generate factor returns (with different volatilities)
    factor_vols = np.array([0.02, 0.015, 0.01, 0.008, 0.005])[:n_factors]
    factors = np.random.randn(n_days, n_factors) * factor_vols
    
    # Add autocorrelation to factors
    for i in range(1, n_days):
        factors[i] = 0.1 * factors[i-1] + 0.95 * factors[i]
    
    # Generate factor loadings (betas) for each stock
    # Use industry structure: stocks in same 'sector' have similar loadings
    n_sectors = 10
    sector_loadings = np.random.randn(n_sectors, n_factors)
    stock_sectors = np.random.choice(n_sectors, n_stocks)
    
    loadings = np.zeros((n_stocks, n_factors))
    for i in range(n_stocks):
        # Sector loading + stock-specific noise
        loadings[i] = sector_loadings[stock_sectors[i]] + np.random.randn(n_factors) * 0.3
    
    # Generate returns: r = loadings @ factors.T + idiosyncratic
    idiosyncratic_vol = np.random.uniform(0.01, 0.03, n_stocks)
    idiosyncratic = np.random.randn(n_days, n_stocks) * idiosyncratic_vol
    
    returns = factors @ loadings.T + idiosyncratic
    
    # Create DataFrame with stock tickers
    tickers = [f'STOCK_{i:03d}' for i in range(n_stocks)]
    dates = pd.date_range('2020-01-01', periods=n_days, freq='B')
    
    df = pd.DataFrame(returns, index=dates, columns=tickers)
    
    return df, factors, loadings, stock_sectors

# Generate data
N_STOCKS = 100
N_DAYS = 1000
N_TRUE_FACTORS = 5

returns_df, true_factors, true_loadings, sectors = generate_factor_returns(
    N_STOCKS, N_DAYS, N_TRUE_FACTORS
)

print(f"Returns shape: {returns_df.shape}")
print(f"True factors shape: {true_factors.shape}")
print(f"True loadings shape: {true_loadings.shape}")
print(f"\nReturn statistics:")
print(returns_df.describe().T.head())

## 2. PCA Baseline

In [ ]:
# Standardize returns
scaler = StandardScaler()
returns_scaled = scaler.fit_transform(returns_df.values)

# PCA
pca = PCA(n_components=10)
pca_factors = pca.fit_transform(returns_scaled)

print("PCA Explained Variance Ratio:")
for i, var in enumerate(pca.explained_variance_ratio_[:10]):
    print(f"  PC{i+1}: {var:.4f} ({sum(pca.explained_variance_ratio_[:i+1]):.4f} cumulative)")

# Plot explained variance
plt.figure(figsize=(10, 4))
plt.bar(range(1, 11), pca.explained_variance_ratio_[:10], alpha=0.7)
plt.plot(range(1, 11), np.cumsum(pca.explained_variance_ratio_[:10]), 'ro-', label='Cumulative')
plt.axhline(y=0.9, color='gray', linestyle='--', label='90% threshold')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Variational Autoencoder (VAE) Architecture

In [ ]:
class VAE(nn.Module):
    """Variational Autoencoder for factor discovery"""
    def __init__(self, input_dim, latent_dim=5, hidden_dims=[64, 32]):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            encoder_layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.LeakyReLU(0.2)
            ])
            prev_dim = h_dim
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Latent space (mean and log-variance)
        self.fc_mu = nn.Linear(hidden_dims[-1], latent_dim)
        self.fc_logvar = nn.Linear(hidden_dims[-1], latent_dim)
        
        # Decoder
        decoder_layers = [nn.Linear(latent_dim, hidden_dims[-1]), nn.LeakyReLU(0.2)]
        for i in range(len(hidden_dims) - 1, 0, -1):
            decoder_layers.extend([
                nn.Linear(hidden_dims[i], hidden_dims[i-1]),
                nn.BatchNorm1d(hidden_dims[i-1]),
                nn.LeakyReLU(0.2)
            ])
        decoder_layers.append(nn.Linear(hidden_dims[0], input_dim))
        self.decoder = nn.Sequential(*decoder_layers)
    
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu, logvar):
        """Reparameterization trick for backprop through sampling"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar, z
    
    def loss_function(self, x, x_recon, mu, logvar, beta=1.0):
        """VAE loss = Reconstruction loss + KL divergence"""
        # Reconstruction loss (MSE)
        recon_loss = F.mse_loss(x_recon, x, reduction='sum')
        
        # KL divergence: -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        
        return recon_loss + beta * kl_loss, recon_loss, kl_loss

# Initialize VAE
LATENT_DIM = 5  # Number of latent factors to discover
vae = VAE(N_STOCKS, latent_dim=LATENT_DIM, hidden_dims=[64, 32]).to(device)
print(f"VAE parameters: {sum(p.numel() for p in vae.parameters()):,}")

## 4. Train VAE

In [ ]:
def train_vae(model, data, epochs=200, batch_size=64, lr=0.001, beta=1.0):
    """Train VAE with annealing beta schedule"""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    data_tensor = torch.tensor(data, dtype=torch.float32).to(device)
    
    losses = {'total': [], 'recon': [], 'kl': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_recon = 0
        total_kl = 0
        n_batches = 0
        
        # Beta annealing: start low, increase to target
        current_beta = min(beta, beta * (epoch / 50))
        
        # Mini-batch training
        indices = torch.randperm(len(data_tensor))
        for i in range(0, len(data_tensor), batch_size):
            batch_idx = indices[i:i+batch_size]
            x_batch = data_tensor[batch_idx]
            
            optimizer.zero_grad()
            x_recon, mu, logvar, z = model(x_batch)
            loss, recon, kl = model.loss_function(x_batch, x_recon, mu, logvar, current_beta)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            total_recon += recon.item()
            total_kl += kl.item()
            n_batches += 1
        
        scheduler.step()
        
        losses['total'].append(total_loss / n_batches)
        losses['recon'].append(total_recon / n_batches)
        losses['kl'].append(total_kl / n_batches)
        
        if (epoch + 1) % 40 == 0:
            print(f"Epoch {epoch+1}: Total={losses['total'][-1]:.2f}, "
                  f"Recon={losses['recon'][-1]:.2f}, KL={losses['kl'][-1]:.2f}")
    
    return losses

# Train
print("Training VAE...")
losses = train_vae(vae, returns_scaled, epochs=200, beta=0.1)

# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(losses['total'])
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Total Loss')
axes[0].set_title('Total Loss')

axes[1].plot(losses['recon'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Reconstruction Loss')
axes[1].set_title('Reconstruction Loss')

axes[2].plot(losses['kl'])
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('KL Divergence')
axes[2].set_title('KL Divergence')

for ax in axes:
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Extract and Analyze Latent Factors

In [ ]:
# Extract latent factors
vae.eval()
with torch.no_grad():
    data_tensor = torch.tensor(returns_scaled, dtype=torch.float32).to(device)
    mu, logvar = vae.encode(data_tensor)
    vae_factors = mu.cpu().numpy()  # Use mean as point estimate

print(f"VAE factors shape: {vae_factors.shape}")

# Compare VAE factors with true factors using correlation
print("\nCorrelation between VAE factors and true factors:")
corr_matrix = np.corrcoef(vae_factors.T, true_factors.T)[:LATENT_DIM, LATENT_DIM:]

plt.figure(figsize=(8, 6))
plt.imshow(np.abs(corr_matrix), cmap='Blues', aspect='auto')
plt.colorbar(label='|Correlation|')
plt.xlabel('True Factor')
plt.ylabel('VAE Factor')
plt.title('Correlation: VAE Factors vs True Factors')
plt.xticks(range(N_TRUE_FACTORS), [f'True {i+1}' for i in range(N_TRUE_FACTORS)])
plt.yticks(range(LATENT_DIM), [f'VAE {i+1}' for i in range(LATENT_DIM)])
for i in range(LATENT_DIM):
    for j in range(N_TRUE_FACTORS):
        plt.text(j, i, f'{corr_matrix[i,j]:.2f}', ha='center', va='center')
plt.tight_layout()
plt.show()

# Best matching (maximum absolute correlation per VAE factor)
print("\nBest factor matches:")
for i in range(LATENT_DIM):
    best_match = np.argmax(np.abs(corr_matrix[i]))
    print(f"  VAE Factor {i+1} -> True Factor {best_match+1} (corr={corr_matrix[i, best_match]:.3f})")

## 6. Visualize Latent Space

In [ ]:
# Compare factor time series
fig, axes = plt.subplots(LATENT_DIM, 2, figsize=(14, 3*LATENT_DIM))

for i in range(LATENT_DIM):
    # VAE factor
    axes[i, 0].plot(vae_factors[:200, i], label=f'VAE Factor {i+1}', alpha=0.8)
    axes[i, 0].set_ylabel('Factor Value')
    axes[i, 0].legend()
    axes[i, 0].grid(True, alpha=0.3)
    
    # Corresponding true factor (best match)
    best_match = np.argmax(np.abs(corr_matrix[i]))
    sign = np.sign(corr_matrix[i, best_match])  # Align signs
    axes[i, 1].plot(sign * true_factors[:200, best_match], 
                   label=f'True Factor {best_match+1}', alpha=0.8, color='orange')
    axes[i, 1].set_ylabel('Factor Value')
    axes[i, 1].legend()
    axes[i, 1].grid(True, alpha=0.3)

axes[-1, 0].set_xlabel('Time')
axes[-1, 1].set_xlabel('Time')
plt.suptitle('VAE Factors vs True Factors (First 200 days)', y=1.02)
plt.tight_layout()
plt.show()

## 7. Factor Loadings Analysis

In [ ]:
# Extract decoder weights as factor loadings
# The first layer of decoder maps latent -> hidden, we want the final mapping
with torch.no_grad():
    # Get effective loadings by passing unit vectors through decoder
    eye = torch.eye(LATENT_DIM).to(device)
    decoded = vae.decode(eye).cpu().numpy()
    vae_loadings = decoded.T  # (n_stocks, latent_dim)

# Compare with true loadings
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# VAE loadings
im1 = axes[0].imshow(vae_loadings, cmap='RdBu', aspect='auto')
axes[0].set_xlabel('VAE Factor')
axes[0].set_ylabel('Stock')
axes[0].set_title('VAE Factor Loadings')
plt.colorbar(im1, ax=axes[0])

# True loadings
im2 = axes[1].imshow(true_loadings, cmap='RdBu', aspect='auto')
axes[1].set_xlabel('True Factor')
axes[1].set_ylabel('Stock')
axes[1].set_title('True Factor Loadings')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

## 8. Reconstruction Quality

In [ ]:
# Compare reconstruction quality: VAE vs PCA
vae.eval()
with torch.no_grad():
    data_tensor = torch.tensor(returns_scaled, dtype=torch.float32).to(device)
    x_recon_vae, _, _, _ = vae(data_tensor)
    x_recon_vae = x_recon_vae.cpu().numpy()

# PCA reconstruction with same number of components
pca_5 = PCA(n_components=LATENT_DIM)
pca_factors_5 = pca_5.fit_transform(returns_scaled)
x_recon_pca = pca_5.inverse_transform(pca_factors_5)

# Calculate reconstruction errors
mse_vae = np.mean((returns_scaled - x_recon_vae) ** 2)
mse_pca = np.mean((returns_scaled - x_recon_pca) ** 2)

# Explained variance
var_original = np.var(returns_scaled)
var_explained_vae = 1 - mse_vae / var_original
var_explained_pca = 1 - mse_pca / var_original

print(f"\nReconstruction Comparison ({LATENT_DIM} factors):")
print(f"  VAE MSE: {mse_vae:.6f}, Variance Explained: {var_explained_vae:.4f}")
print(f"  PCA MSE: {mse_pca:.6f}, Variance Explained: {var_explained_pca:.4f}")

# Visualize reconstruction for a few stocks
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
stocks_to_show = [0, 25, 50, 75]
days_to_show = 100

for ax, stock_idx in zip(axes.flat, stocks_to_show):
    ax.plot(returns_scaled[:days_to_show, stock_idx], label='Original', alpha=0.8)
    ax.plot(x_recon_vae[:days_to_show, stock_idx], label='VAE', alpha=0.8)
    ax.plot(x_recon_pca[:days_to_show, stock_idx], label='PCA', alpha=0.8)
    ax.set_title(f'Stock {stock_idx}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Reconstruction Comparison: Original vs VAE vs PCA')
plt.tight_layout()
plt.show()

## 9. Generate New Samples from Latent Space

In [ ]:
# Sample from latent space to generate new return scenarios
vae.eval()
n_samples = 500

with torch.no_grad():
    # Sample from standard normal (prior)
    z_samples = torch.randn(n_samples, LATENT_DIM).to(device)
    generated_returns = vae.decode(z_samples).cpu().numpy()

# Compare statistics
print("Return Statistics Comparison:")
print(f"{'Metric':<20} {'Original':<15} {'Generated':<15}")
print("-" * 50)
print(f"{'Mean':<20} {returns_scaled.mean():.6f}     {generated_returns.mean():.6f}")
print(f"{'Std':<20} {returns_scaled.std():.6f}     {generated_returns.std():.6f}")
print(f"{'Min':<20} {returns_scaled.min():.6f}    {generated_returns.min():.6f}")
print(f"{'Max':<20} {returns_scaled.max():.6f}     {generated_returns.max():.6f}")

# Plot distribution comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(returns_scaled.flatten(), bins=50, alpha=0.7, density=True, label='Original')
axes[0].hist(generated_returns.flatten(), bins=50, alpha=0.7, density=True, label='Generated')
axes[0].set_xlabel('Return')
axes[0].set_ylabel('Density')
axes[0].set_title('Return Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Cross-sectional correlation comparison
orig_corr = np.corrcoef(returns_scaled[:, :20].T)
gen_corr = np.corrcoef(generated_returns[:, :20].T)

axes[1].scatter(orig_corr.flatten(), gen_corr.flatten(), alpha=0.5, s=10)
axes[1].plot([-1, 1], [-1, 1], 'r--', label='Perfect match')
axes[1].set_xlabel('Original Correlation')
axes[1].set_ylabel('Generated Correlation')
axes[1].set_title('Cross-Sectional Correlations')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Variational Autoencoder (VAE)**: Probabilistic encoder-decoder for factor discovery
2. **Latent Factor Extraction**: Using encoder to map returns to factor space
3. **Comparison with PCA**: VAE can capture non-linear relationships
4. **Generative Capability**: Sample new return scenarios from learned distribution

### Key Insights:
- VAE learns factors that correlate with true underlying factors
- Regularized latent space allows for meaningful interpolation/sampling
- Non-linear decoder can capture complex stock-factor relationships

### Extensions to Try:
- Use real stock returns from yfinance
- Add conditional VAE (CVAE) with sector/industry labels
- Implement beta-VAE for more disentangled factors
- Use temporal VAE (T-VAE) to capture time dynamics